In [10]:
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re
import random

In [27]:
# 1

def convert_to_hot(syl_word):
    hot = []
    i = 0
    while  i < len(syl_word):
        if i == len(syl_word) - 1:
            hot += [1]
            return hot
        if syl_word[i+1] == '-':
            hot += [2]
            i += 2
        else:
            hot += [1]
            i += 1
    return hot

In [28]:
# 2
# data prep on Ox

training_data_size = 20000
e2i_vocab_ortho = []
e2i_vocab_ipa = []

x_tr_ortho = []
x_tr_ipa = []
y_tr = []

x_val_ortho = []
x_val_ipa = []
y_val = []

orig_file = open('data/post_clean.txt')

orig_data = orig_file.readlines()
orig_file.close()
orig_data = [line.strip('\n') for line in orig_data]
random.shuffle(orig_data)
data_eng = [line.split(';')[0].lower() for line in orig_data]
data_syl = [line.split(';')[1].lower() for line in orig_data]


max_encoder_len_ortho = len(max(data_eng, key=len))
print(max_encoder_len_ortho)

20


In [33]:
# 3

y_tr = []

for line in data_eng:
    for c in line:
        if c not in e2i_vocab_ortho:
            e2i_vocab_ortho += [c]
            
e2i_ortho = dict((a,i) for i,a in enumerate(e2i_vocab_ortho, 1))

for line in data_eng[:training_data_size]:
    converted = []
    for c in line:
        converted += [e2i_ortho[c]]
    x_tr_ortho += [converted]
    
x_tr_ortho = pad_sequences(x_tr_ortho, maxlen=20, padding='post')

for line in data_syl[:training_data_size]:
    y_tr += [convert_to_hot(line)]

y_tr = pad_sequences(y_tr, maxlen=20, padding='post')


x_tr_file_ortho = open('data/ox/x_tr_ortho.pkl','wb')
pickle.dump(x_tr_ortho, x_tr_file_ortho)
x_tr_file_ortho.close()

y_tr_file = open('data/ox/y_tr.pkl','wb')
pickle.dump(y_tr, y_tr_file)
y_tr_file.close()

e2i_ortho_file = open('data/ox/e2i_ortho.pkl','wb')
pickle.dump(e2i_ortho, e2i_ortho_file)
e2i_ortho_file.close()

In [35]:
# 4
# validation data processing

x_val_ortho = []
x_val_ipa = []
y_val = []

for line in data_eng[training_data_size:]:
    converted = []
    for c in line:
        converted += [e2i_ortho[c]]
    x_val_ortho += [converted]
    
x_val_ortho = pad_sequences(x_val_ortho, maxlen=20, padding='post')

for line in data_syl[training_data_size:]:
    y_val += [convert_to_hot(line)]

y_val = pad_sequences(y_val, maxlen=20, padding='post')


x_val_file_ortho = open('data/ox/x_val_ortho.pkl','wb')
pickle.dump(x_val_ortho, x_val_file_ortho)
x_val_file_ortho.close()

y_val_file = open('data/ox/y_val.pkl','wb')
pickle.dump(y_val, y_val_file)
y_val_file.close()

In [18]:
# cross referencing
# ¥

with open('data/mhyph.txt', encoding="ISO-8859-1") as mhyph:
    moby = mhyph.readlines()
    moby = [line.strip('\n') for line in moby]
    moby = [(line.replace('¥','').replace('-',''), line.replace('¥','-')) for line in moby if ' ' not in line]
    moby_dict = dict(moby)
    
with open('data/scraped_clean_clean.txt') as ox:
    ox_list = ox.readlines()
    ox_list = [line.strip('\n') for line in ox_list]
    ox_list = [(line.split(';')[0], line.split(';')[1]) for line in ox_list]
    ox_dict = dict(ox_list)

In [28]:
post_checked = []
mismatches = []

print(len(ox_dict.keys()))

for word in ox_dict.keys():
    try:
        if ox_dict[word] == moby_dict[word]:
            post_checked += [(word, ox_dict[word])]
        else:
            if word in moby_dict.keys():
                mismatches += [(ox_dict[word], moby_dict[word])]
    except:
        post_checked += [(word, ox_dict[word])]
            
print(len(post_checked))
print(list(ox_dict.keys())[:5])
print(post_checked[:5])
print(mismatches[:5])

27644
26503
['a', 'abandon', 'abandoned', 'ability', 'able']
[('a', 'a'), ('abandon', 'a-ban-don'), ('abandoned', 'a-ban-doned'), ('ability', 'a-bil-i-ty'), ('able', 'a-ble')]
[('aged', 'a-ged'), ('a-na-lyze', 'an-a-lyse'), ('a-na-ly-sis', 'a-nal-y-sis'), ('a-ni-mal', 'an-i-mal'), ('at-tach', 'at-ta-ch')]


In [38]:
for i in range(0, len(data_syl)):
    if data_syl[i].lower() == 'images':
        print(data_eng[i])

In [43]:
# grabbing data for syllable percentage estimate

num_pos = 0
num_neg = 0
total = 0
for word in y_tr:
    for c in word:
        if c == 2:
            num_pos += 1
            total += 1
        elif c == 1:
            num_neg += 1
            total += 1
        

In [44]:
print(num_pos)
print(num_neg)
print(total)
print(float(num_pos)/float(total))

32871
125387
158258
0.20770513970857712
